<a href="https://colab.research.google.com/github/fajrulsantoso/Pcvk_2025-2026/blob/main/Selamat_Datang_di_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Mount Google Drive

In [23]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
train_dir = '/content/drive/MyDrive/DATASETCM/train'
test_dir  = '/content/drive/MyDrive/DATASETCM/test'


3. Import library preprocessing

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


4. Membuat ImageDataGenerator (Preprocessing Utama)

In [26]:
train_datagen = ImageDataGenerator(
    rescale=1./255,          # normalisasi piksel
    rotation_range=20,       # rotasi random
    width_shift_range=0.2,   # geser horizontal
    height_shift_range=0.2,  # geser vertikal
    shear_range=0.2,         # shear
    zoom_range=0.2,          # zoom
    horizontal_flip=True,    # flip horizontal
    fill_mode='nearest',     # pengisian piksel kosong
    validation_split=0.2     # 20% data train → validation
)


✅ b. Untuk Validation (tanpa augmentasi)

In [27]:
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


✅ c. Untuk Testing

In [28]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [29]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 667 images belonging to 5 classes.


5. Load dataset dari folder

In [30]:
val_data = val_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 165 images belonging to 5 classes.


Validation Set (20%)

In [31]:
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 328 images belonging to 5 classes.


In [32]:
print("Train classes:", train_data.class_indices)
print("Total train images:", train_data.samples)

print("Validation images:", val_data.samples)
print("Test images:", test_data.samples)


Train classes: {'bawang_merah': 0, 'bawang_putih': 1, 'brokoli': 2, 'bunga_kol': 3, 'wortel': 4}
Total train images: 667
Validation images: 165
Test images: 328


Import librar

In [33]:
import tensorflow as tf
from tensorflow.keras import layers, models


Bangun Arsitektur CNN

In [34]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),          # mencegah overfitting
    layers.Dense(5, activation='softmax')   # 5 kelas
])


Compile model

In [35]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Lihat rangkuman arsitektur

In [36]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,997 (18.42 MB)

 Trainable params: 4,828,997 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

Training model



In [44]:
images, labels = next(train_data)

print("Label shape:", labels.shape)
print(labels[0])


Label shape: (32, 5)
[0. 1. 0. 0. 0.]


In [ ]:
Runtime → Restart runtime
